In [1]:
from tensorflow import keras
import tensorflow as tf
import os

In [2]:
def greenBlock(inputTensor, channel, strides):
    """
    Burada basemodel üzerinden kullanılacak katmanlar şunlardır:
    112*112*64: block_1_expand_relu (112*112*96)
    56*56*128: block_3_expand_relu (56*56*144)
    28*28*256: block_6_expand_relu (28*28*192)
    14*14*512: block_13_expand_relu (14*14*576)
    7*7*1024: out_relu (7*7*1280)
    """
    tensor = keras.layers.DepthwiseConv2D(
        kernel_size = 3,
        strides = 1,
        padding = "same",
        use_bias = False
    )(inputTensor)
    tensor = keras.layers.BatchNormalization()(tensor)
    tensor = keras.layers.ReLU()(tensor)
    tensor = keras.layers.Conv2D(
        filters = channel,
        kernel_size = 1,
        padding = "same",
        use_bias = False
    )(tensor)
    tensor = keras.layers.BatchNormalization()(tensor)
    skipConnection = keras.layers.ReLU()(tensor)
    tensor = keras.layers.MaxPooling2D(strides)(skipConnection)
    return tensor, skipConnection

In [3]:
def orangeBlock(inputTensor, channel):
    tensor = keras.layers.DepthwiseConv2D(
        kernel_size=3,
        padding="same"
    )(inputTensor)
    tensor = keras.layers.Conv2D(
        filters = channel,
        kernel_size = 1,
        padding="same"
    )(tensor)
    relu = keras.layers.ReLU()(tensor)
    return relu

In [4]:
def redBlock(inputTensor):
    tensor = keras.layers.Conv2D(
        filters = 2,
        kernel_size = 1,
        padding="same"
    )(inputTensor)
    tensor = keras.layers.Softmax()(tensor)
    return tensor

In [5]:
def blueBlock(inputTensor):
    # flatten katmanı ekleyelim
    # tensor = keras.layers.Flatten()(inputTensor)
    tensor = keras.layers.Conv2D(
        filters = 32,
        strides = 2,
        kernel_size = 3,
        padding="same"
    )(inputTensor)
    return tensor

In [6]:
def yellowBlock(inputLayer1, inputLayer2, channel):
    layer1 = keras.layers.Conv2D(
        filters = channel,
        kernel_size = 1,
        padding = "same",
        activation = "relu"
    )(inputLayer1)
    layer2 = keras.layers.UpSampling2D(
        size = 2,
        interpolation = "bilinear"
    )(inputLayer2)
    tensor = keras.layers.Add()([layer1, layer2])
    return tensor

In [7]:
def createModel(input_boyutu=(224, 224, 3)):
    """
    keras.applications.MobileNetV2 omurgası kullanarak derin öğrenme modeli oluşturur (Encoder-Decoder mimarisi).

    Bu model, girdi görüntüsünden bir maske (segmentasyon maskesi) üretmek için tasarlanmıştır.
    Encoder bölümü olarak önceden eğitilmiş keras.applications.MobileNetV2 modeli kullanılır.
    Decoder bölümü ise yukarı örnekleme ve basit konvolüsyon blokları ile maskeyi oluşturur.
    Atlama bağlantıları (skip connections) encoder"dan decoder"a detay bilgisi taşır.
    """
    baseModel = keras.applications.MobileNetV2(
        include_top=False,
        weights="imagenet",
        input_shape=input_boyutu
    )
    baseModel.trainable = False

    # MobileNetV2'den alınacak katmanların isimleri ve çıktıları
    # layer_names = [
    #     'block_1_expand_relu',   # ~112x112
    #     'block_3_expand_relu',   # ~56x56
    #     'block_6_expand_relu',   # ~28x28
    #     'block_13_expand_relu'   # ~14x14
    # ]
    # mobileNetSublayers = {name: baseModel.get_layer(name).output for name in layer_names}

    # Encoder çıkışını al (en derin katman)
    # inputName = 'out_relu'
    # tensor = baseModel.get_layer(inputName).output # 7x7x1280

    # ENCODER
    inputTensor = baseModel.input
    tensor = blueBlock(inputTensor) # 112*112*32
    print(tensor.shape)
    tensor, skipConnection112 = greenBlock(tensor, 64, 1) # 112*112*64
    print(tensor.shape)
    tensor, _ = greenBlock(tensor, 128, 2) # 56*56*128
    print(tensor.shape)
    tensor, skipConnection56 = greenBlock(tensor, 128, 1) # 56*56*128
    print(tensor.shape)
    tensor, _ = greenBlock(tensor, 256, 2) # 28*28*256
    print(tensor.shape)
    tensor, skipConnection28 = greenBlock(tensor, 256, 1) # 28*28*256
    print(tensor.shape)
    tensor, _ = greenBlock(tensor, 512, 2) # 14*14*512
    print(tensor.shape)
    tensor, _ = greenBlock(tensor, 512, 1) # 14*14*512  
    print(tensor.shape)
    # tensor, _ = greenBlock(tensor, 512, 1) # 14*14*512
    # print(tensor.shape)
    # tensor, _ = greenBlock(tensor, 512, 1) # 14*14*512
    # print(tensor.shape)
    tensor, _ = greenBlock(tensor, 512, 1) # 14*14*512
    print(tensor.shape)
    tensor, skipConnection14 = greenBlock(tensor, 512, 1) # 14*14*512
    print(tensor.shape)
    tensor, _ = greenBlock(tensor, 1024, 2) # 7*7*1024
    print(tensor.shape)
    tensor, _ = greenBlock(tensor, 1024, 1) # 7*7*1024
    print(tensor.shape)
    print("--------------------------")
    
    # DECODER
    tensor = yellowBlock(skipConnection14, tensor, 1024) # 14*14*1024
    print(tensor.shape)
    tensor = orangeBlock(tensor, 64) # 14*14*64
    print(tensor.shape)
    tensor = yellowBlock(skipConnection28, tensor, 64) # 28*28*64
    print(tensor.shape)
    tensor = orangeBlock(tensor, 64) # 28*28*64
    print(tensor.shape)
    tensor = yellowBlock(skipConnection56, tensor, 64) # 56*56*64
    print(tensor.shape)
    tensor = orangeBlock(tensor, 64) # 56*56*64
    print(tensor.shape)
    tensor = yellowBlock(skipConnection112, tensor, 64) # 112*112*64
    print(tensor.shape)
    tensor = orangeBlock(tensor, 64) # 112*112*64
    print(tensor.shape)
    tensor = keras.layers.UpSampling2D(
        size = 2,
        interpolation = "bilinear"
    )(tensor) # 224*224*64
    print(tensor.shape)
    tensor = orangeBlock(tensor, 64) # 224*224*64
    print(tensor.shape)
    tensor = redBlock(tensor) # 224*224*2
    print(tensor.shape)
    return keras.Model(inputs=baseModel.input, outputs=tensor)


In [8]:
def resizeDataset(imageDirectory, maskDirectory, targetSize=(224, 224), batchSize=8):
    resizeAndRescale = keras.Sequential([
        keras.layers.Resizing(targetSize[0], targetSize[1]),
        keras.layers.Rescaling(1./255)
    ])

    imageDataset = keras.utils.image_dataset_from_directory(
        imageDirectory,
        batch_size=batchSize,
        seed=42,
        image_size=targetSize,
        label_mode = None,
        color_mode = "rgb",
        shuffle = False
    )

    maskDataset = keras.utils.image_dataset_from_directory(
        maskDirectory,
        batch_size=batchSize,
        seed=42,
        image_size=targetSize,
        label_mode = None,
        color_mode = "grayscale",
        shuffle = False
    )

    imageDataset = imageDataset.map(lambda x: resizeAndRescale(x))
    maskDataset = maskDataset.map(lambda x: resizeAndRescale(x))
    # Maskeleri normalleştir ve uygun formata dönüştür
    def convert_to_binary_masks(mask):        
        # 0.5 eşik değeri ile ikili maske oluştur
        mask = tf.cast(mask > 0.5, tf.float32)
        
        # Her piksel için [arka plan, saç] kanallarını oluştur
        background = 1 - mask
        hair = mask
        
        # İki kanalı birleştir: [arka plan, saç]
        one_hot_mask = tf.concat([background, hair], axis=-1)
        return one_hot_mask
    
    maskDataset = maskDataset.map(convert_to_binary_masks)

    # Veri setlerini birleştir
    dataset = tf.data.Dataset.zip((imageDataset, maskDataset))
    dataset = dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset


In [9]:
model = createModel()
model.compile(
    optimizer=keras.optimizers.Adadelta(learning_rate=1.0, rho=0.95, epsilon=1e-7),
    loss="categorical_crossentropy",
    metrics=[
        "accuracy", "precision", "recall",
        keras.metrics.MeanIoU(num_classes=2),
        keras.metrics.IoU(num_classes=2, target_class_ids=[1]),
        "categorical_accuracy", 
        "mse"
    ]
)

(None, 112, 112, 32)
(None, 112, 112, 64)
(None, 56, 56, 128)
(None, 56, 56, 128)
(None, 28, 28, 256)
(None, 28, 28, 256)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 7, 7, 1024)
(None, 7, 7, 1024)
--------------------------
(None, 14, 14, 1024)
(None, 14, 14, 64)
(None, 28, 28, 64)
(None, 28, 28, 64)
(None, 56, 56, 64)
(None, 56, 56, 64)
(None, 112, 112, 64)
(None, 112, 112, 64)
(None, 224, 224, 64)
(None, 224, 224, 64)
(None, 224, 224, 2)


In [10]:
# model.summary()

In [12]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=r"E:\Ben\TRYazilim\hair_analysis_test_project\src\hair_type_segmentation\deneme.keras",
        monitor = "val_loss",
        save_best_only = True,
        mode = "min",
        verbose = 1,
        save_weights_only = False
    )
]

In [13]:
trainDataset = resizeDataset(r"E:\Ben\TRYazilim\Figaro-1k\train\image", r"E:\Ben\TRYazilim\Figaro-1k\train\mask")
testDataset = resizeDataset(r"E:\Ben\TRYazilim\Figaro-1k\test\image", r"E:\Ben\TRYazilim\Figaro-1k\test\mask")


Found 840 files.
Found 0 files.


ValueError: No images found in directory E:\Ben\TRYazilim\Figaro-1k\train\mask. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

In [ ]:
model.fit(
    trainDataset,
    epochs=60,
    batch_size=4,
    validation_data=testDataset,
    callbacks=callbacks
)

Epoch 1/60
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6506 - categorical_accuracy: 0.6506 - io_u_1: 0.0000e+00 - loss: 0.6346 - mean_io_u_1: 0.2500 - mse: 0.2220 - precision: 0.6506 - recall: 0.6506
Epoch 1: val_loss improved from inf to 0.63374, saving model to model/model_best_best.keras
125/125 ━━━━━━━━━━━━━━━━━━━━ 416s 3s/step - accuracy: 0.6508 - categorical_accuracy: 0.6508 - io_u_1: 0.0000e+00 - loss: 0.6344 - mean_io_u_1: 0.2500 - mse: 0.2219 - precision: 0.6508 - recall: 0.6508 - val_accuracy: 0.6799 - val_categorical_accuracy: 0.6799 - val_io_u_1: 0.0000e+00 - val_loss: 0.6337 - val_mean_io_u_1: 0.2500 - val_mse: 0.2208 - val_precision: 0.6799 - val_recall: 0.6799
Epoch 2/60
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6678 - categorical_accuracy: 0.6678 - io_u_1: 0.0000e+00 - loss: 0.6000 - mean_io_u_1: 0.2500 - mse: 0.2067 - precision: 0.6678 - recall: 0.6678
Epoch 2: val_loss did not improve from 0.63374
125/125 ━━━━━━━━━━━━━━━━━━━━ 392s 3s/step - accur

In [ ]:
model.save("modelepoch60.keras")